# GA & PSO Comparison

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

## Load GA & PSO Results

In [ ]:
# Paths to results
GA_RESULTS_PATH = "experiments/results/ga/"
PSO_RESULTS_PATH = "experiments/results/pso/"

# Load results
ga_results = pd.read_csv(GA_RESULTS_PATH)
pso_results = pd.read_csv(PSO_RESULTS_PATH)

# Add algorithm labels
ga_results["algorithm"] = "GA"
pso_results["algorithm"] = "PSO"

# Combine results
combined_results = pd.concat([ga_results, pso_results], ignore_index=True)

## Plot Boxplots for Fitness & Evaluations

In [ ]:
# Plot fitness boxplots
plt.figure(figsize=(12, 8))
sns.boxplot(x="num_companies", y="sharpe_ratio", hue="algorithm", data=combined_results, palette={"GA": "blue", "PSO": "orange"})
plt.title("Fitness Comparison by Portfolio Size")
plt.xlabel("Number of Assets")
plt.ylabel("Sharpe Ratio")
plt.legend(title="Algorithm")
plt.show()

# Plot evaluations boxplots
plt.figure(figsize=(12, 8))
sns.boxplot(x="num_companies", y="n_evaluations", hue="algorithm", data=combined_results, palette={"GA": "blue", "PSO": "orange"})
plt.title("Evaluations Comparison by Portfolio Size")
plt.xlabel("Number of Assets")
plt.ylabel("Number of Evaluations")
plt.legend(title="Algorithm")
plt.show()

## Wilcoxon Signed-Rank Test for Pairwise Comparison

In [ ]:
alpha = 0.05

# Iterate over unique portfolio sizes
for num_companies in combined_results["num_companies"].unique():
    print(f"\nPortfolio Size: {num_companies}")

    # Filter results for the current portfolio size
    ga_data = combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "GA")]
    pso_data = combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "PSO")]

    # Perform Wilcoxon test for fitness
    stat, p_value = wilcoxon(ga_data["sharpe_ratio"], pso_data["sharpe_ratio"])
    print(f"Fitness - GA Mean: {ga_data['sharpe_ratio'].mean():.4f}, PSO Mean: {pso_data['sharpe_ratio'].mean():.4f}")
    print("The difference in fitness is statistically significant." if p_value < alpha else "The difference in fitness is NOT statistically significant.")

    # Perform Wilcoxon test for evaluations
    stat, p_value = wilcoxon(ga_data["n_evaluations"], pso_data["n_evaluations"])
    print(f"Evaluations - GA Mean: {ga_data['n_evaluations'].mean():.4f}, PSO Mean: {pso_data['n_evaluations'].mean():.4f}")
    print("The difference in evaluations is statistically significant." if p_value < alpha else "The difference in evaluations is NOT statistically significant.")


## Friedman Test for Overall Algorithm Comparison

In [ ]:
from scipy.stats import friedmanchisquare

# Prepare data for Friedman test
fitness_data = [
    combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "GA")]["sharpe_ratio"].values
    for num_companies in combined_results["num_companies"].unique()
] + [
    combined_results[(combined_results["num_companies"] == num_companies) & (combined_results["algorithm"] == "PSO")]["sharpe_ratio"].values
    for num_companies in combined_results["num_companies"].unique()
]

# Perform Friedman test for fitness
stat, p_value = friedmanchisquare(*fitness_data)
print("Friedman Test for Fitness")
print(f"Statistic: {stat:.4f}, P-Value: {p_value:.4f}")
print("The difference in fitness across algorithms is statistically significant." if p_value < alpha else "The difference in fitness across algorithms is NOT statistically significant.")